In [26]:
from datetime import datetime
from subgrounds.subgraph import SyntheticField
from subgrounds.subgrounds import Subgrounds
import pandas as pd

In [27]:
sg = Subgrounds()
snapshot = sg.load_api('https://hub.snapshot.org/graphql')

In [28]:
snapshot.Proposal.datetime = SyntheticField(
  lambda timestamp: str(datetime.fromtimestamp(timestamp)),
  SyntheticField.STRING,
  snapshot.Proposal.end,
)

In [29]:
proposals = snapshot.Query.proposals(
  orderBy='created',
  orderDirection='desc',
  first=10000,
  where=[
    snapshot.Proposal.space == 'olympusdao.eth',
    snapshot.Proposal.state == 'closed'
    ##snapshot.Proposal.title == 'OIP-18: Reward rate framework and reduction',
  ]
)

In [30]:
proposals_snapshots = sg.query_df([
    proposals.title,
    proposals.id,
    proposals.body,
    proposals.scores,
    proposals.scores_total
])

In [31]:
proposals_choices = sg.query(proposals.choices)

TypeError: 'FieldPath' object is not iterable

In [25]:
proposals_choices = pd.DataFrame(proposals_choices, columns = ['option_a', 'option_b', 'option_c', 'option_d', 'option_e','option_f','option_g'])

NameError: name 'proposals_choices' is not defined

In [ ]:
olympus_governance_view = pd.concat([proposals_snapshots,proposals_choices], axis=1)

In [ ]:
##let's view the output just to make sure
olympus_governance_view

In [ ]:
path = '"C:\Users\ianfe\PycharmProjects\snapshot_governance\olympus-proposals.csv"'
olympus_governance_view.to_csv(path, index = False)

In [ ]:
total_proposals = len(olympus_governance_view)
total_proposals

In [ ]:
proposal_id = olympus_governance_view.iloc[0,1]

In [ ]:
print(proposal_id)

In [ ]:

vote_tracker = snapshot.Query.votes(
orderBy = 'created',
orderDirection='desc',
first=10000,
where=[
  snapshot.Vote.proposal == proposal_id
]
)

In [ ]:
voting_snapshots_list = sg.query_df([
    vote_tracker.id,
    vote_tracker.voter,
    vote_tracker.created,
    vote_tracker.choice,
    vote_tracker.vp
])

In [ ]:
voting_snapshots_list

In [ ]:
voting_snapshots_list['Proposal'] = proposal_id
print(voting_snapshots_list)

In [ ]:
x=0
while x <total_proposals:
##print(olympus_governance_view.iloc[2,1])
    proposal_id = olympus_governance_view.iloc[x,1]

    vote_tracker = snapshot.Query.votes(
    orderBy = 'created',
    orderDirection='desc',
    first=10000,
    where=[
      snapshot.Vote.proposal == proposal_id
    ]
    )
    voting_snapshots = sg.query_df([
    vote_tracker.id,
    vote_tracker.voter,
    vote_tracker.created,
    vote_tracker.choice,
    vote_tracker.vp
    ])

    voting_snapshots['Proposal'] = proposal_id
    voting_snapshots_list=pd.concat([voting_snapshots_list, voting_snapshots])
    ##voting_snapshots['Proposal'] = proposal_id

#    pd.concat([voting_snapshots_list, voting_snapshots])
    x=x+1

In [ ]:
len(voting_snapshots_list)

In [ ]:
path = '"C:\Users\ianfe\PycharmProjects\snapshot_governance\olympus-votes.csv"'
voting_snapshots_list.to_csv(path, index = False)